In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
import os, os.path
import sys
sys.path.insert(0,'../processing/')
sys.path.insert(0,'../')
from datasets import sythtextprovider
import load_batch
from nets import txtbox_300, textbox_common, np_methods
#from processing import image_processing
from image_processing2 import *
from processing import txt_preprocessing
import tf_utils
import time
slim = tf.contrib.slim
import load_batch
import numpy as np
import skimage.io as skio
from processing import tf_image
import cv2

In [2]:
isess = tf.InteractiveSession()

In [ ]:
with tf.Graph().as_default(): 
    text_net = txtbox_300.TextboxNet()
    text_shape = text_net.params.img_shape
    print 'text_shape '+  str(text_shape)
    text_anchors = text_net.anchors(text_shape)

    b_image, b_glocalisations, b_gscores = \
        load_batch.get_batch('../data/sythtext/',
                             3,
                             1,
                             text_shape,
                             text_net,
                             text_anchors,
                             2,
                             file_pattern = '*.tfrecord',
                             is_training = True,
                             shuffe = False)
    localisations, logits, end_points = \
                        text_net.net(b_image, is_training=True, use_batch=False)
    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        with slim.queues.QueueRunners(sess):
            for i in xrange(1):
                

In [3]:
scope='text_box_300'
inputs = tf.placeholder(shape=(None,300,300,3),dtype=tf.float32)
end_points = {}
with tf.variable_scope(scope, 'text_box_300', [inputs], reuse=None):
    # Original VGG-16 blocks.
    net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
    #net = batch_norm(net,is_training=is_training, scope='batch1',use_batch=use_batch)
    end_points['conv1'] = net
    net = slim.max_pool2d(net, [2, 2], scope='pool1')
    # Block 2.
    net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
    #net = batch_norm(net,is_training=is_training, scope='batch2',use_batch=use_batch)
    end_points['conv2'] = net # 150,150 128
    net = slim.max_pool2d(net, [2, 2], scope='pool2')
    # Block 3. # 75 75 256
    net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
    #net = batch_norm(net,is_training=is_training, scope='batch3',use_batch=use_batch)
    end_points['conv3'] = net
    net = slim.max_pool2d(net, [2, 2], scope='pool3',padding='SAME')
    # Block 4. # 38 38 512
    net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
    #net = batch_norm(net,is_training=is_training, scope='batch4',use_batch=use_batch)
    end_points['conv4'] = net
    net = slim.max_pool2d(net, [2, 2], scope='pool4')
    # Block 5. # 19 19 512
    net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
    #net = batch_norm(net,is_training=is_training, scope='batch5',use_batch=use_batch)
    end_points['conv5'] = net
    net = slim.max_pool2d(net, [3, 3], stride=1, scope='pool5',padding='SAME')

    # Additional SSD blocks.
    # Block 6: let's dilate the hell out of it!
    net = slim.conv2d(net, 1024, [3, 3], scope='conv6')
    #net = batch_norm(net,is_training=is_training, scope='batch6',use_batch=use_batch)
    end_points['conv6'] = net
    #net = tf.layers.dropout(net, rate=dropout_keep_prob, training = is_training)
    # Block 7: 1x1 conv. Because the fuck.
    net = slim.conv2d(net, 1024, [1, 1], scope='conv7')
    #net = batch_norm(net,is_training=is_training, scope='batch7',use_batch=use_batch)
    end_points['conv7'] = net
    #net = tf.layers.dropout(net, rate=dropout_keep_prob, training = is_training)
    # Block 8/9/10/11: 1x1 and 3x3 convolutions stride 2 (except lasts).


In [5]:
with tf.variable_scope(scope, 'text_box_300', [net], reuse=None):
    end_point = 'conv8'
    with tf.variable_scope(end_point):
        net = slim.conv2d(net, 256, [1, 1], scope='conv1x1')
        #net = custom_layers.pad2d(net, pad=(1, 1))
        net = slim.conv2d(net, 512, [3, 3], stride=2, scope='conv3x3')
        #net = batch_norm(net,is_training=is_training,use_batch=use_batch)
    end_points[end_point] = net
    end_point = 'conv9'
    with tf.variable_scope(end_point):
        net = slim.conv2d(net, 128, [1, 1], scope='conv1x1')
        #net = custom_layers.pad2d(net, pad=(1, 1))
        net = slim.conv2d(net, 256, [3, 3], stride=2, scope='conv3x3')
        #net = batch_norm(net,is_training=is_training,use_batch=use_batch)
    end_points[end_point] = net
    end_point = 'conv10'
    with tf.variable_scope(end_point):
        net = slim.conv2d(net, 128, [1, 1], scope='conv1x1')
        net = slim.conv2d(net, 256, [3, 3],stride=2, scope='conv3x3')
        #net = batch_norm(net,is_training=is_training,use_batch=use_batch)
    end_points[end_point] = net
    end_point = 'global'
    with tf.variable_scope(end_point):
        #net = slim.avg_pool2d(net, [3,3], scope='pool6', padding = 'VALID')
        net = slim.conv2d(net, 128, [1, 1], scope='conv1x1')
        net = slim.conv2d(net, 256, [3, 3], scope='conv3x3', padding='VALID')
        #net = batch_norm(net,is_training=is_training,use_batch=use_batch)
    end_points[end_point] = net

In [6]:
end_points


{'conv1': <tf.Tensor 'text_box_300/conv1/conv1_2/Relu:0' shape=(?, 300, 300, 64) dtype=float32>,
 'conv10': <tf.Tensor 'text_box_300_2/conv10/conv3x3/Relu:0' shape=(?, 3, 3, 256) dtype=float32>,
 'conv2': <tf.Tensor 'text_box_300/conv2/conv2_2/Relu:0' shape=(?, 150, 150, 128) dtype=float32>,
 'conv3': <tf.Tensor 'text_box_300/conv3/conv3_3/Relu:0' shape=(?, 75, 75, 256) dtype=float32>,
 'conv4': <tf.Tensor 'text_box_300/conv4/conv4_3/Relu:0' shape=(?, 38, 38, 512) dtype=float32>,
 'conv5': <tf.Tensor 'text_box_300/conv5/conv5_3/Relu:0' shape=(?, 19, 19, 512) dtype=float32>,
 'conv6': <tf.Tensor 'text_box_300/conv6/Relu:0' shape=(?, 19, 19, 1024) dtype=float32>,
 'conv7': <tf.Tensor 'text_box_300/conv7/Relu:0' shape=(?, 19, 19, 1024) dtype=float32>,
 'conv8': <tf.Tensor 'text_box_300_2/conv8/conv3x3/Relu:0' shape=(?, 10, 10, 512) dtype=float32>,
 'conv9': <tf.Tensor 'text_box_300_2/conv9/conv3x3/Relu:0' shape=(?, 5, 5, 256) dtype=float32>,
 'global': <tf.Tensor 'text_box_300_2/global/co